In [1]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex
import pandas as pd
import numpy as np
import xlsxwriter

# Calibration
## Estimation of the model parameters
quetzal_germany is being estimated using [PandasBiogeme](https://biogeme.epfl.ch/). This notebook estimates calibration parameters for the model's utility functions.
- Documentation and reference: [Bierlaire, M. (2020). A short introduction to PandasBiogeme. Technical report TRANSP-OR 200605. Transport and Mobility Laboratory, ENAC, EPFL.](https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf)
- Tutorial: https://www.youtube.com/watch?v=OiM94B8WayA

### Model formulation
The model consists of systematic utility functions, one for each mode. They comprise an alternaive-specific constant (ASC), a distance-dependent part with travel time and cost summarised as generalised cost (GC), and a cost damping function F

> V_i = ASC + F(GC(T, C), b_gc_i)

Index i marks the demand group. I = {'commuting' (1), 'education' (2), 'shopping/medical' (3), 'business' (4), 'private' (6)}

Note: The cost variable already includes subscriptions

In [2]:
input_path = '../input/'
model_path = '../model/'

### Prepare the database

In [3]:
df = pd.read_csv(input_path + 'transport_demand/calibration_inter-cellular_trips_MiD2017.csv')
df = df[['cost_rail_short', 'cost_rail_long', 'cost_car', 'cost_coach', 'cost_bus', 'cost_walk', 'cost_air',
         'time_rail_short', 'time_rail_long', 'time_car', 'time_coach', 'time_bus', 'time_walk', 'time_air',
#         'accessibility_rail', 'accessibility_car', 'accessibility_coach', 'accessibility_bus',
#         'accessibility_walk', 'accessibility_air',
         'mode_model', 'purpose_vp', 'car_avail', 'distance']]
df.columns = ['C_RAIL_S', 'C_RAIL_L', 'C_CAR', 'C_COACH', 'C_BUS', 'C_NON_MOTOR', 'C_AIR',
              'T_RAIL_S', 'T_RAIL_L', 'T_CAR', 'T_COACH', 'T_BUS', 'T_NON_MOTOR', 'T_AIR',
#              'AC_RAIL', 'AC_CAR', 'AC_COACH', 'AC_BUS', 'AC_NON_MOTOR', 'AC_AIR',
              'MODE', 'PURPOSE', 'CAR_AV', 'DIST']

In [4]:
inf = 1000
df = df.replace({np.inf:inf})

In [5]:
df.describe()

,C_RAIL_S,C_RAIL_L,C_CAR,C_COACH,C_BUS,C_NON_MOTOR,C_AIR,T_RAIL_S,T_RAIL_L,T_CAR,T_COACH,T_BUS,T_NON_MOTOR,T_AIR,MODE,PURPOSE,CAR_AV,DIST
count,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000,86522.0,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000,86522.000000
mean,9.089089,89.518449,6.196420,5.471111,2.728534,0.0,979.919668,398.204316,617.093366,50.080036,593.927378,593.243781,745.093888,984.874365,5.569300,3.634763,0.963905,39.105289
std,12.220584,55.993352,5.701401,2.504218,1.150123,0.0,134.318501,426.921907,428.646573,55.691218,420.274327,464.280071,416.084206,101.592929,1.361315,2.054412,0.190937,59.366383
min,0.000000,19.000000,0.014477,5.000000,0.000000,0.0,43.000000,3.000000,1.000000,1.000000,10.000000,2.000000,1.000000,55.000000,1.000000,1.000000,0.000000,0.010000
25%,3.681500,21.181381,2.827377,5.000000,2.740000,0.0,1000.000000,67.800000,120.450000,25.000000,136.100000,66.700000,114.933333,1000.000000,6.000000,1.000000,1.000000,13.780000
50%,5.895000,139.000000,5.207430,5.000000,2.740000,0.0,1000.000000,110.825000,1000.000000,35.000000,1000.000000,1000.000000,1000.000000,1000.000000,6.000000,3.000000,1.000000,23.750000
75%,9.879300,139.000000,7.916824,5.000000,4.000000,0.0,1000.000000,1000.000000,1000.000000,55.000000,1000.000000,1000.000000,1000.000000,1000.000000,6.000000,6.000000,1.000000,39.900000
max,100.000000,139.000000,106.812632,51.081500,4.000000,0.0,1000.000000,1000.000000,1000.000000,623.016667,1000.000000,1000.000000,1000.000000,1000.000000,7.000000,6.000000,9.000000,902.500000


In [6]:
# Scale time to hours
df['T_RAIL_S_S'] = df['T_RAIL_S'] / 60
df['T_RAIL_L_S'] = df['T_RAIL_L'] / 60
df['T_CAR_S'] = df['T_CAR'] / 60
df['T_COACH_S'] = df['T_COACH'] / 60
df['T_BUS_S'] = df['T_BUS'] / 60
df['T_AIR_S'] = df['T_AIR'] / 60
df['T_NON_MOTOR_S'] = df['T_NON_MOTOR'] / 60

In [7]:
# Make car availability binary
df['CAR_AV'] = df['CAR_AV'].replace({9:0})

In [8]:
# Remove trips where mode is car but the car availability is zero
# because it irritates the MLE algorithm
mask = ((df['MODE']==6) & (df['CAR_AV']==0))
print('Share of car trips dropped: {}'.format(
    len(df.loc[mask])/len(df.loc[df['MODE']==6])))
df = df.loc[~mask]

Share of car trips dropped: 0.01385512882839086


In [9]:
# Add PT availabilities
df['RAIL_S_AV'] = (df['T_RAIL_S']!=inf).astype(int)
df['RAIL_L_AV'] = (df['T_RAIL_L']!=inf).astype(int)
df['COACH_AV'] = (df['T_COACH']!=inf).astype(int)
df['BUS_AV'] = (df['T_BUS']!=inf).astype(int)
df['AIR_AV'] = (df['T_AIR']!=inf).astype(int)
df['NON_MOTOR_AV'] = (df['T_NON_MOTOR']!=inf).astype(int)

In [10]:
# Remove trips where cost are infinity
# Share of drops per mode must be equal, otherwise the calibration is skewed
# Ignore air trips because this dataset has too few observations anyways
max_drop_ratio = 0
lengths = []
for mode, col in zip(range(1,5), ['C_RAIL_S', 'C_RAIL_L', 'C_COACH', 'C_BUS']):
    drops = df.loc[((df['MODE']==mode) & (df[col]==inf))].index
    lengths = lengths + list(df.loc[drops, 'DIST'])
    if len(drops) > 0: print('mode ' + str(mode) + ': ' + str(len(drops)) + ' drops')
    max_drop_ratio = max(len(drops) / len(df.loc[df['MODE']==mode]), max_drop_ratio)
print('max_drop_ratio: ' + str(max_drop_ratio))
if max_drop_ratio > 0:
    print('Dropped trips length mean: {}; min: {}; max: {}'.format(
        sum(lengths)/len(lengths), min(lengths), max(lengths)))

max_drop_ratio: 0.0


In [11]:
# Drop trips with infinite cost
if max_drop_ratio > 0:
    df = df.loc[~((df['MODE']==2) & (df['C_RAIL_L']==inf))]
    # Don't drop air trips because there are only a few
    for m in [1, 3, 4, 6]:
        n_drops = int(max_drop_ratio * len(df.loc[df['MODE']==m]))
        df = df.drop(df.loc[(df['MODE']==m) & (df['DIST']<max(lengths)) &
                            (df['DIST']>min(lengths))].sample(n_drops).index)
        print('mode ' + str(m) + ': ' + str(n_drops) + ' drops')
    print('New number of observations: ' + str(len(df.index)))

In [12]:
# Merge rail short and long because people percieve it the same
#df['MODE'] = df['MODE'].replace({2:1})

### Model variables
All columns are variables. DefineVariable creates a new column in the database.

#### Cost damping

Many modelling studies have shown that cost damping is required in order to flatten the tail of time/cost elasticities, i.e. decrease the impact of long distances on choice results to prevent from overestimation of time/cost parameters. Cost damping represents the property of decreasing marginal utility. It is commonly approached with Box-Cox transformations of generalised cost (usually defined as the sum of travel time and travel expenditures divided by the value of time). Daly (2010) proposes a hybrid function as a sum of the linear term and the narural logarithm of the same. However, the linear term still dominates cost on long distances. Rich (2015), main developer of the Danish National Transport Model, proposes a more complex spline function which successfully manages cost damping and even outperforms the Box-Cox transformation in terms of stability of elasticities.

In [13]:
# Using the hybrid linear-logarithmic version requires a new column
# for the logarithm of time in minutes
'''df['T_RAIL_S'] = np.log(df['T_RAIL'])
df['T_CAR_S'] = np.log(df['T_CAR'])
df['T_COACH_S'] = np.log(df['T_COACH'])
df['T_BUS_S'] = np.log(df['T_BUS'])
df['T_AIR_S'] = np.log(df['T_AIR'])
df['T_NON_MOTOR_S'] = np.log(df['T_NON_MOTOR'])'''
print('This functional form performes poorly')

This functional form performes poorly


#### Generalised cost

Both, travel time and monetary cost should be affected by cost damping measures. It is logical to define a generalised cost term `GC` with dimension of time units. This requires definition or estimation of values of time, in order to rescale monetary units, for all segments. Usually, the value of time (VoT) is distance-dependent. In the case of Germany, VoT can be taken from research undertaken within the Federal Government's transport study "Bundesverkehrswegeplan 2030": Axhausen et al. 2015. Ermittlung von Bewertungsansätzen für Reisezeiten und Zuverlässigkeit auf der Basis eines Modells für modale Verlagerungen im nicht-gewerblichen und gewerblichen Personenverkehr für die Bundesverkehrswegeplanung

In [14]:
# VoT from literature, distance-dependent, see cal19
VoT = pd.read_csv(input_path + 'vot.csv', header=[0,1], index_col=0)
VoT.sample(2)

,root,Fz1,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,...,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,Fz4,Fz6
,all,all,all,all,all,all,PT,PT,PT,PT,...,air,air,air,air,car,car,car,car,car,car
700,20.6600,19.7600,9.18,11.9,27.2300,15.5500,15.5500,11.8900,13.48,16.3200,...,72.810,72.810,177.1800,40.9400,20.4300,19.3800,12.1700,25.8300,22.620,15.5500
766,21.5642,20.6114,9.18,11.9,29.0252,16.1506,16.2034,12.3256,13.48,16.7556,...,80.367,80.367,206.2398,44.8604,21.3078,20.1918,12.6254,26.8794,23.709,16.1506


In [15]:
# Make distance integer
df['DIST'] = df['DIST'].apply(int)

In [16]:
# Generate generalised cost
VoT = VoT.to_dict()
df['GC_RAIL_S'] = df['T_RAIL_S_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_S'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_L'] = df['T_RAIL_L_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_L'], df['DIST'], df['PURPOSE'])]
df['GC_COACH'] = df['T_COACH_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                    for c,d,p in zip(df['C_COACH'], df['DIST'], df['PURPOSE'])]
df['GC_BUS'] = df['T_BUS_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                for c,d,p in zip(df['C_BUS'], df['DIST'], df['PURPOSE'])]
df['GC_AIR'] = df['T_AIR_S'] + [c / VoT[('Fz'+str(p), 'air')][d]
                                for c,d,p in zip(df['C_AIR'], df['DIST'], df['PURPOSE'])]
df['GC_CAR'] = df['T_CAR_S'] + [c / VoT[('Fz'+str(p), 'car')][d]
                                for c,d,p in zip(df['C_CAR'], df['DIST'], df['PURPOSE'])]
df['GC_NON_MOTOR'] = df['T_NON_MOTOR_S']

In [17]:
# Create the initial database and make columns global variables
database = db.Database('MiD2017', df.copy())
globals().update(database.variables)
database.getSampleSize()

85496

### Estimation parameters

In [18]:
asc_rail_s = ex.Beta('asc_rail_s', 0, None, None, 0)
asc_rail_l = ex.Beta('asc_rail_l', 0, None, None, 0)
asc_coach = ex.Beta('asc_coach', 0, None, None, 0)
asc_bus = ex.Beta('asc_bus', 0, None, None, 0)
asc_air = ex.Beta('asc_air', 0, None, None, 0)
asc_car = ex.Beta('asc_car', 0, None, None, 1)
asc_non_motor = ex.Beta('asc_non_motor', 0, None, None, 0)

In [19]:
b_t = ex.Beta('b_t', 0, None, None, 0)
b_c = ex.Beta('b_c', 0, None, None, 0)
b_ac = ex.Beta('b_ac', 0, None, None, 0)

In [20]:
# non-linear time component
b_t2 = ex.Beta('b_t2', 0, None, None, 0)

In [21]:
# generalised cost function parameter
b_gc = ex.Beta('b_gc', 0, None, None, 0)

In [22]:
# Parameters for the nested logit structure
mu_pt = ex.Beta('mu_pt', 1, 1, 10, 0)

### Utility functions

In [23]:
'''# Aggregated formulation
V_RAIL = asc_rail + b_t * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR_S

# Aggregated formulation with non-linear perception of travel time
V_RAIL = asc_rail + b_t * T_RAIL + b_t2 * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH + b_t2 * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS + b_t2 * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR + b_t2 * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR + b_t2 * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR + b_t2 * T_NON_MOTOR_S'''
print('Deprected')

Deprected


In [24]:
# The cost damping function
def spline(x, beta=b_gc, Q=3, c0=0, c1=20, c2=40, c3=np.inf):
    alpha = [0, -beta/2*ex.Power(ex.log(c1),3),
             -beta/2*ex.log(c1)*(3*ex.Power(ex.log(c2),2)+ex.Power(ex.log(c1),2))] # for Q=3
    theta = [1, 3/2*ex.log(c1), 3*ex.log(c1)*ex.log(c2)] # for Q=3
    def component(x, q):
        return beta*theta[q-1]*ex.Power(ex.log(x),Q-q+1) + alpha[q-1]
    if x < c1:
        return component(x,1)
    elif x < c2:
        return component(x,2)
    elif x >= c2:
        return component(x,3)
    #return sum([component(x, q) if x>=c[q-1] and x<c[q] else 0 for q in range(1,Q+1)])

In [25]:
# Aggregated formulation with damped generalised cost
V_RAIL_S = asc_rail_s + spline(GC_RAIL_S)# + b_ac * AC_RAIL
V_RAIL_L = asc_rail_l + spline(GC_RAIL_L)# + b_ac * AC_RAIL
V_COACH = asc_coach + spline(GC_COACH)# + b_ac * AC_COACH
V_BUS = asc_bus + spline(GC_BUS)# + b_ac * AC_BUS
V_AIR = asc_air + spline(GC_AIR)# + b_ac * AC_AIR
V_CAR = asc_car + spline(GC_CAR)
V_NON_MOTOR = asc_non_motor + spline(GC_NON_MOTOR)

### Run the estimation

In [26]:
# Define level of verbosity
logger = message.bioMessage()
#logger.setSilent()
logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

In [27]:
# Map modes to utility functions
V = {1:V_RAIL_S,
     2:V_RAIL_L,
     3:V_COACH,
     4:V_BUS,
     5:V_AIR,
     6:V_CAR,
     7:V_NON_MOTOR}

In [28]:
# Map the availability of alternatives with MODE as key
# PT is always available
av = {1:RAIL_S_AV,
      2:RAIL_L_AV,
      3:COACH_AV,
      4:BUS_AV,
      5:AIR_AV,
      6:CAR_AV,
      7:NON_MOTOR_AV}

In [29]:
# Mode nests as tuples with nest name and dictionary where
# alternative IDs are mapped to alpha values. Missing ID's alpha is zero
# Alternatively use lists with mode ID without alpha
pt = mu_pt, [1, 2, 3, 4]
air = 1, [5]
car = 1, [6]
walk = 1, [7]
nests = pt, air, car, walk

In [30]:
# Choose the multinomial logit model
mnl = models.loglogit(V, av, MODE)

In [31]:
model_mnl = bio.BIOGEME(database, mnl)
model_mnl.modelName = 'MNL'

In [32]:
results_mnl = model_mnl.estimate()

In [33]:
results = results_mnl.getEstimatedParameters()
for key, val in results_mnl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-5.477395,0.27856,-19.6632,0,0.419035,-13.0714,0
asc_bus,-2.504357,0.0201494,-124.289,0,0.0198242,-126.328,0
asc_coach,-4.698154,0.0698242,-67.2855,0,0.0695165,-67.5833,0
asc_non_motor,-3.056215,0.0294697,-103.707,0,0.033647,-90.8318,0
asc_rail_l,-1.897484,0.0401797,-47.225,0,0.0459015,-41.3382,0
asc_rail_s,-1.838658,0.0156222,-117.695,0,0.0179283,-102.556,0
b_gc,-0.535919,0.00734867,-72.9273,0,0.017748,-30.196,0
Number of estimated parameters,7.000000,,,,,,
Sample size,85496.000000,,,,,,
Excluded observations,0.000000,,,,,,


In [34]:
# Write results to a file
writer = pd.ExcelWriter(input_path + 'estimation_results.xlsx', engine='xlsxwriter')

In [35]:
results.to_excel(writer, sheet_name=model_mnl.modelName)

In [36]:
# Choose the logarithmic nested logit model
nl = models.lognested(V, av, nests, MODE)

In [37]:
# Nested Logit
model_nl = bio.BIOGEME(database, nl)
model_nl.modelName = 'NL'

In [38]:
results_nl = model_nl.estimate()

In [39]:
results = results_nl.getEstimatedParameters()
for key, val in results_nl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-4.800468,0.26238,-18.2958,0,0.368429,-13.0296,0
asc_bus,-2.241667,0.0198273,-113.059,0,0.0244131,-91.8224,0
asc_coach,-3.735327,0.0629708,-59.3184,0,0.0834108,-44.7823,0
asc_non_motor,-2.928021,0.0286852,-102.074,0,0.0319475,-91.651,0
asc_rail_l,-1.564035,0.0330461,-47.3289,0,0.041942,-37.2904,0
asc_rail_s,-1.724557,0.0146192,-117.965,0,0.0178927,-96.3833,0
b_gc,-0.447381,0.00768259,-58.2331,0,0.0172742,-25.8987,0
mu_pt,1.494400,0.0278822,53.5969,0,0.0489469,30.531,0
Number of estimated parameters,8.000000,,,,,,
Sample size,85496.000000,,,,,,


In [40]:
results.to_excel(writer, sheet_name=model_nl.modelName)

In [41]:
# Run the estimation by purpose
results = []
for p in [1,2,3,4,6]:
    database = db.Database('MiD2017', df.copy())
    database.remove(PURPOSE!=p)
    print('Sample size for purpose {}: {}'.format(p, database.getSampleSize()))
    model = bio.BIOGEME(database, nl) # Choose the model formulation
    model.modelName = 'NL_Fz' + str(p) # Name it
    results.append(model.estimate()) # Estimation
    output = results[-1].getEstimatedParameters()
    # Add results to the Excel file
    for key, val in results[-1].getGeneralStatistics().items():
        output.loc[key] = [val[0], val[1]] + ['' for i in range(len(output.columns)-2)]
    output.to_excel(writer, sheet_name=model.modelName)

Sample size for purpose 1: 22969
Sample size for purpose 2: 3693
Sample size for purpose 3: 21276
Sample size for purpose 4: 4923
Sample size for purpose 6: 32635


In [42]:
writer.save()

In [43]:
# Generate LaTeX code
file = open(input_path + 'estimation_results_LaTeX_code.txt', 'w')
for r in results:
    file.write(r.getLaTeX())
file.close()